In [ ]:
# Cell 1: Initial setup - Mount Drive, Install Libraries, Force Restart
# *** IMPORTANT: Run this as your FIRST cell after "Disconnect and delete runtime" and reconnecting ***

from google.colab import drive
drive.mount('/content/drive')

print("Uninstalling existing versions to ensure a clean slate...")
# Uninstall all related packages comprehensively.
!pip uninstall -y torch torchvision torchaudio numpy transformers accelerate safetensors tokenizers sentencepiece packaging langchain langchain_community chromadb h5py sentence_transformers pillow streamlit # <<< ADDED STREAMLIT HERE

print("\nInstalling latest versions of core libraries, allowing pip to resolve for NumPy 2.x...")

# Install PyTorch for Colab's CUDA version.
!pip install -qU torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

# Install latest versions of other libraries.
!pip install -qU transformers accelerate sentence-transformers langchain langchain_community chromadb h5py packaging pandas openpyxl streamlit # <<< ADDED STREAMLIT HERE
!pip install -qU "unstructured[all-docs]"

# --- Verification of installed versions (now safe to import after installation) ---
import torch
import numpy as np
try:
    import transformers
    import accelerate
    import sentence_transformers
    import langchain
    import packaging
    import pandas
    import streamlit # <<< ADDED STREAMLIT HERE
except ImportError as e:
    print(f"Error importing modules for verification (this might happen before restart): {e}")

print(f"\nPyTorch CUDA available: {torch.cuda.is_available()}")
print(f"PyTorch CUDA version: {torch.version.cuda}")
print(f"NVIDIA GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'N/A'}")
print(f"Installed torch version: {torch.__version__}")
print(f"Installed numpy version: {np.__version__}")
if 'transformers' in locals():
    print(f"Installed transformers version: {transformers.__version__}")
if 'accelerate' in locals():
    print(f"Installed accelerate version: {accelerate.__version__}")
if 'sentence_transformers' in locals():
    print(f"Installed sentence-transformers version: {sentence_transformers.__version__}")
if 'langchain' in locals():
    print(f"Installed langchain version: {langchain.__version__}")
if 'packaging' in locals():
    print(f"Installed packaging version: {packaging.__version__}")
if 'pandas' in locals():
    print(f"Installed pandas version: {pandas.__version__}")
if 'streamlit' in locals(): # <<< ADDED STREAMLIT HERE
    print(f"Installed streamlit version: {streamlit.__version__}")
# --- End Verification ---

print("\nInitial installations complete. Forcing runtime restart to apply all changes.")
import os
os._exit(0) # This will crash the runtime.

Mounted at /content/drive
Uninstalling existing versions to ensure a clean slate...
Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: torchaudio 2.6.0+cu124
Uninstalling torchaudio-2.6.0+cu124:
  Successfully uninstalled torchaudio-2.6.0+cu124
Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
Found existing installation: transformers 4.53.1
Uninstalling transformers-4.53.1:
  Successfully uninstalled transformers-4.53.1
Found existing installation: accelerate 1.8.1
Uninstalling accelerate-1.8.1:
  Successfully uninstalled accelerate-1.8.1
Found existing installation: safetensors 0.5.3
Uninstalling safetensors-0.5.3:
  Successfully uninstalled safetensors-0.5.3
Found existing installation

In [1]:
# Cell 2: Task 2 - Create and Populate ChromaDB
# Run this *after* Cell 1 has crashed and you've restarted the session.

from google.colab import drive
import os
import pandas as pd
from langchain_community.document_loaders import CSVLoader, UnstructuredExcelLoader # Import relevant loaders
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.docstore.document import Document
import warnings

warnings.filterwarnings('ignore') # Suppress warnings

# Re-mount Google Drive (essential after any restart/reset)
drive.mount('/content/drive')
print(f"DEBUG: Google Drive mounted at /content/drive") # DEBUG PRINT

# --- Configuration (MUST MATCH other cells) ---
DATA_DIR = '/content/drive/MyDrive/'
# IMPORTANT: Adjust this to your actual data file name and type
DATA_FILE_NAME = 'customer_complaints.csv' # or 'customer_complaints.xlsx'
DATA_FILE_PATH = os.path.join(DATA_DIR, DATA_FILE_NAME)

VECTOR_STORE_DIR = '/content/drive/MyDrive/vector_store'
CHROMA_PERSIST_DIR = os.path.join(VECTOR_STORE_DIR, 'chroma_db_credi_trust')
EMBEDDING_MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
COLLECTION_NAME = "credi_trust_complaints" # <--- THIS MUST BE CONSISTENT

# Ensure the parent directory for the vector store exists
os.makedirs(VECTOR_STORE_DIR, exist_ok=True)
print(f"Ensuring parent directory for vector store exists: {VECTOR_STORE_DIR}")

# --- Step 1: Load your raw data ---
documents = []
print(f"Attempting to load data from: {DATA_FILE_PATH}")
if not os.path.exists(DATA_FILE_PATH):
    print(f"ERROR: Data file not found at {DATA_FILE_PATH}.")
    print("Creating dummy data for demonstration. In a real scenario, please upload your data file to Google Drive.")
    # Create dummy data if file is not found
    dummy_data = {
        'complaint_id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
        'product': ['Credit Card', 'Personal Loan', 'Savings Account', 'Online Banking', 'BNPL', 'Credit Card', 'Money Transfer', 'Personal Loan', 'Savings Account', 'BNPL'],
        'issue': ['Unauthorized charge', 'Delayed approval', 'High fees', 'Login problem', 'Late fee dispute', 'Fraudulent activity', 'Delayed transfer', 'Interest rate change', 'Account closure', 'Payment issue'],
        'narrative': [
            "Customer reported an unauthorized charge of $150 on their credit card statement. The bank did not resolve it.",
            "Applied for a personal loan, but the approval process has been severely delayed for over two months. No communication.",
            "Monthly maintenance fees on my savings account are excessively high and were not disclosed clearly during sign-up.",
            "Unable to log into online banking for several days. Tried password reset multiple times, but it never works.",
            "Disputed a late fee on a Buy Now, Pay Later installment, as payment was submitted on time according to my records.",
            "Found suspicious activity and fraudulent charges on my credit card. I reported it immediately.",
            "A money transfer I initiated to my family member was delayed by over a week, causing significant distress.",
            "The interest rate on my personal loan was unexpectedly increased without clear notification, making payments difficult.",
            "My savings account was closed without any prior notice or explanation from the bank, locking my funds.",
            "Experiencing recurring issues with my Buy Now, Pay Later payments not processing correctly, leading to late fees."
        ]
    }
    df = pd.DataFrame(dummy_data)
    # Convert dummy data to LangChain Documents
    for index, row in df.iterrows():
        page_content = row['narrative']
        metadata = {
            'complaint_id': row.get('complaint_id', index),
            'product': row.get('product', 'Unknown'),
            'issue': row.get('issue', 'Unknown')
        }
        documents.append(Document(page_content=page_content, metadata=metadata))
    print(f"Loaded {len(documents)} dummy documents.")

else:
    # Use appropriate loader based on your file type
    if DATA_FILE_NAME.endswith('.csv'):
        loader = CSVLoader(file_path=DATA_FILE_PATH, encoding="utf-8") # Adjust encoding if needed
        documents = loader.load()
    elif DATA_FILE_NAME.endswith('.xlsx'):
        loader = UnstructuredExcelLoader(file_path=DATA_FILE_PATH, mode="elements") # Using unstructured for Excel
        documents = loader.load()
    else:
        print("ERROR: Unsupported data file format. Please use .csv or .xlsx.")
        exit() # Stop if format not supported

    print(f"Loaded {len(documents)} documents from {DATA_FILE_NAME}.")
    if not documents:
        print("WARNING: No documents were loaded from the file. Please check file content and format.")
        exit()

# Step 2: Split documents into chunks
print("Splitting documents into chunks...")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(documents)
print(f"Split {len(documents)} documents into {len(chunks)} chunks.")
if not chunks:
    print("CRITICAL ERROR: No chunks were created. This means either no data was loaded or splitting failed.")
    exit() # Stop execution if no chunks to embed

# Step 3: Initialize Embedding Model
print(f"Initializing embedding model: {EMBEDDING_MODEL_NAME}...")
embedding_model = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL_NAME)
print("Embedding model initialized.")

# Step 4: Create and Persist ChromaDB
print(f"Attempting to create/update and persist ChromaDB at: {CHROMA_PERSIST_DIR} with collection '{COLLECTION_NAME}'...")
try:
    # Delete existing directory content to ensure a clean rebuild (optional but good for debugging)
    if os.path.exists(CHROMA_PERSIST_DIR):
        import shutil
        print(f"Clearing existing ChromaDB directory: {CHROMA_PERSIST_DIR}")
        shutil.rmtree(CHROMA_PERSIST_DIR)
        os.makedirs(CHROMA_PERSIST_DIR) # Recreate empty directory

    # Use from_documents to create the vector store from chunks
    vector_store = Chroma.from_documents(
        chunks,
        embedding_model,
        persist_directory=CHROMA_PERSIST_DIR,
        collection_name=COLLECTION_NAME # <--- THIS IS CRITICAL AND MUST BE CONSISTENT
    )
    # Explicitly persist the database. This is vital!
    vector_store.persist()
    print("ChromaDB vector store created/updated and persisted successfully.")

    # Verify the count of documents in the collection
    count = vector_store._collection.count()
    print(f"Current number of documents in collection '{COLLECTION_NAME}': {count}")
    if count == 0:
        print("CRITICAL WARNING: The collection still appears to be empty after creation. Data was not added.")
    else:
        print(f"SUCCESS: ChromaDB collection '{COLLECTION_NAME}' contains {count} documents.")

except Exception as e:
    print(f"FATAL ERROR: Failed to create or persist ChromaDB: {e}")
    print("Please check permissions, Google Drive space, and ensure the embedding model is working.")

print("\n--- Task 2: ChromaDB Population Complete ---")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
DEBUG: Google Drive mounted at /content/drive
Ensuring parent directory for vector store exists: /content/drive/MyDrive/vector_store
Attempting to load data from: /content/drive/MyDrive/customer_complaints.csv
ERROR: Data file not found at /content/drive/MyDrive/customer_complaints.csv.
Creating dummy data for demonstration. In a real scenario, please upload your data file to Google Drive.
Loaded 10 dummy documents.
Splitting documents into chunks...
Split 10 documents into 10 chunks.
Initializing embedding model: sentence-transformers/all-MiniLM-L6-v2...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embedding model initialized.
Attempting to create/update and persist ChromaDB at: /content/drive/MyDrive/vector_store/chroma_db_credi_trust with collection 'credi_trust_complaints'...
Clearing existing ChromaDB directory: /content/drive/MyDrive/vector_store/chroma_db_credi_trust
ChromaDB vector store created/updated and persisted successfully.
Current number of documents in collection 'credi_trust_complaints': 10
SUCCESS: ChromaDB collection 'credi_trust_complaints' contains 10 documents.

--- Task 2: ChromaDB Population Complete ---


In [2]:
# Cell 3: Write app.py (Streamlit Application Code)
%%writefile app.py
import streamlit as st
import os
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain_community.llms import HuggingFacePipeline
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import numpy as np

# --- Configuration (MUST match your vector store path) ---
VECTOR_STORE_DIR = '/content/drive/MyDrive/vector_store'
CHROMA_PERSIST_DIR = os.path.join(VECTOR_STORE_DIR, 'chroma_db_credi_trust')

# --- REMOVE THIS LINE: ---
# from google.colab import drive
# drive.mount('/content/drive')
# print(f"DEBUG: Google Drive mounted at /content/drive") # DEBUG PRINT
# -------------------------

# --- 1. Initialize Embedding Model and Vector Store (cached) ---
@st.cache_resource
def initialize_rag_components():
    """Initializes and caches the embedding model and vector store."""
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    embedding_model = HuggingFaceEmbeddings(model_name=model_name)
    st.success(f"Initialized embedding model: {model_name}")
    print(f"DEBUG: Initialized embedding model: {model_name}") # DEBUG PRINT

    vector_store = None # Initialize to None
    try:
        # Check if the persist directory actually contains files
        if not os.path.exists(CHROMA_PERSIST_DIR) or not os.path.exists(os.path.join(CHROMA_PERSIST_DIR, 'chroma.sqlite3')):
            st.warning(f"ChromaDB persist directory '{CHROMA_PERSIST_DIR}' does not contain 'chroma.sqlite3' or does not exist.")
            print(f"DEBUG: ChromaDB persist directory '{CHROMA_PERSIST_DIR}' does not contain 'chroma.sqlite3' or does not exist.") # DEBUG PRINT
            st.warning("Please ensure Task 2 was completed successfully and the vector store exists at the specified Google Drive path.")
            return None, None # Return None to indicate failure to load

        vector_store = Chroma(
            persist_directory=CHROMA_PERSIST_DIR,
            embedding_function=embedding_model,
            collection_name="credi_trust_complaints"
        )
        st.success(f"Loaded ChromaDB vector store from: {CHROMA_PERSIST_DIR}")
        print(f"DEBUG: Loaded ChromaDB vector store from: {CHROMA_PERSIST_DIR}") # DEBUG PRINT

        # Verify collection count
        try:
            collection_count = vector_store._collection.count()
            st.success(f"ChromaDB collection 'credi_trust_complaints' contains {collection_count} documents.")
            print(f"DEBUG: ChromaDB collection 'credi_trust_complaints' contains {collection_count} documents.") # DEBUG PRINT
            if collection_count == 0:
                st.warning("Warning: The ChromaDB collection appears to be empty. Retrieval will fail.")
                print("DEBUG: Warning: The ChromaDB collection appears to be empty. Retrieval will fail.") # DEBUG PRINT

        except Exception as count_e:
            st.error(f"Error checking ChromaDB collection count: {count_e}")
            print(f"DEBUG: Error checking ChromaDB collection count: {count_e}") # DEBUG PRINT


        return embedding_model, vector_store
    except Exception as e:
        st.error(f"Error loading ChromaDB vector store: {e}")
        print(f"DEBUG: Error loading ChromaDB vector store: {e}") # DEBUG PRINT
        st.warning("Please ensure Task 2 was completed successfully and the vector store exists at the specified Google Drive path.")
        return None, None

# --- 2. Initialize Language Model (LLM) (cached) ---
@st.cache_resource
def initialize_llm():
    """Initializes and caches the language model."""
    model_id = "google/flan-t5-small" # Using the smaller model as per your Cell 2 config

    device = "cuda" if torch.cuda.is_available() else "cpu"
    st.success(f"CUDA available: {torch.cuda.is_available()}. Using device: {device}")
    print(f"DEBUG: CUDA available: {torch.cuda.is_available()}. Using device: {device}") # DEBUG PRINT

    try:
        tokenizer = AutoTokenizer.from_pretrained(model_id)
        model = AutoModelForSeq2SeqLM.from_pretrained(model_id, torch_dtype=torch.float32).to(device)

        st.success(f"Model '{model_id}' moved to device: {device}")
        print(f"DEBUG: Model '{model_id}' moved to device: {device}") # DEBUG PRINT

        pipe = pipeline(
            "text2text-generation",
            model=model,
            tokenizer=tokenizer,
            max_new_tokens=256,
            device=0 if device == "cuda" else -1,
            do_sample=True,
            temperature=0.7,
            top_p=0.9
        )
        llm = HuggingFacePipeline(pipeline=pipe)
        st.success(f"Initialized LLM: {model_id}")
        print(f"DEBUG: Initialized LLM: {model_id}") # DEBUG PRINT
        return llm
    except Exception as e:
        st.error(f"Error initializing LLM '{model_id}': {e}")
        print(f"DEBUG: Error initializing LLM '{model_id}': {e}") # DEBUG PRINT
        st.warning("Possible solutions: Check available RAM/VRAM, try a smaller model, or ensure necessary dependencies are installed.")
        return None

# --- 3. Prompt Engineering (cached) ---
@st.cache_resource
def get_rag_prompt_template():
    """Returns the RAG prompt template."""
    template = """You are a financial analyst assistant for CrediTrust. Your task is to answer questions about customer complaints. Use the following retrieved complaint excerpts to formulate your answer.
If the context doesn't contain the answer, state that you don't have enough information.

Context:
{context}

Question: {question}
Answer:
"""
    prompt = PromptTemplate(template=template, input_variables=["context", "question"])
    st.success("Defined RAG prompt template.")
    print("DEBUG: Defined RAG prompt template.") # DEBUG PRINT
    return prompt

# --- 4. Retriever Function ---
def retrieve_chunks(question: str, vector_store, k: int = 5):
    """Retrieves relevant text chunks from the vector store."""
    if vector_store is None:
        st.error("Vector store not initialized. Cannot retrieve chunks.")
        print("DEBUG: Vector store not initialized. Cannot retrieve chunks in retrieve_chunks.") # DEBUG PRINT
        return [], []

    st.info(f"Retrieving top {k} chunks for question: '{question}'...")
    print(f"DEBUG: Retrieving top {k} chunks for question: '{question}'...") # DEBUG PRINT

    results = []
    try:
        results = vector_store.similarity_search(question, k=k)
        print(f"DEBUG: Raw similarity_search results (first result, if any): {results[0].page_content[:100] + '...' if results else 'None'}") # DEBUG PRINT
    except Exception as search_e:
        st.error(f"Error during similarity search: {search_e}")
        print(f"DEBUG: Error during similarity search: {search_e}") # DEBUG PRINT
        return [], []

    context_texts = [doc.page_content for doc in results]

    source_documents = []
    for doc in results:
        source_documents.append({
            "content": doc.page_content,
            "metadata": doc.metadata
        })
    st.success(f"Retrieved {len(context_texts)} chunks.")
    print(f"DEBUG: Retrieved {len(context_texts)} chunks.") # DEBUG PRINT
    return context_texts, source_documents

# --- 5. Generator Function ---
def generate_response(llm, prompt_template, question: str, context_texts: list):
    """Generates an answer using the LLM and prompt template."""
    # Corrected: Use 'is None' for checking None in Python
    if llm is None:
        st.error("LLM not initialized. Cannot generate response.")
        print("DEBUG: LLM not initialized. Cannot generate response in generate_response.") # DEBUG PRINT
        return "Error: Language Model not available."

    context_str = "\n\n".join(context_texts)
    print(f"DEBUG: Context provided to LLM (first 200 chars): {context_str[:200]}...") # DEBUG PRINT
    final_prompt = prompt_template.format(context=context_str, question=question)
    print(f"DEBUG: Final prompt sent to LLM (first 500 chars): {final_prompt[:500]}...") # DEBUG PRINT

    st.info("Generating response with LLM...")
    try:
        response = llm.invoke(final_prompt)
        print(f"DEBUG: LLM Raw Response (first 200 chars): {response[:200]}...") # DEBUG PRINT

        if isinstance(response, str):
            # For HuggingFacePipeline, the response might be a string directly
            generated_text = response
        elif isinstance(response, list) and len(response) > 0 and 'generated_text' in response[0]:
            generated_text = response[0]['generated_text']
        else:
            print(f"DEBUG: Unexpected LLM response format: {response}")
            return "Could not generate a coherent answer due to unexpected LLM response format."

        # Post-processing: remove prompt if echoed by model
        # Using a more robust check to avoid issues if prompt isn't at the very start
        if generated_text.startswith(final_prompt):
            generated_text = generated_text[len(final_prompt):].strip()
        # Remove "Answer:" prefix if present from the model's output
        if generated_text.lower().startswith("answer:"):
            generated_text = generated_text[len("answer:"):].strip()
        return generated_text
    except Exception as e:
        st.error(f"Error during LLM inference: {e}")
        print(f"DEBUG: Error during LLM inference: {e}") # DEBUG PRINT
        return "An error occurred while generating the answer. Please try again."

# --- Main Streamlit Application ---

st.set_page_config(page_title="CrediTrust RAG Chatbot", layout="wide")
st.title("🤝 CrediTrust Financial Complaint Assistant (RAG)")
st.markdown("Ask questions about customer complaints and get answers with sources!")

# Initialize RAG components once using st.cache_resource
embedding_model, vector_store = initialize_rag_components()
llm = initialize_llm()
prompt_template = get_rag_prompt_template()

# Initialize chat history in session state
if "messages" not in st.session_state:
    st.session_state.messages = []

# Display chat messages from history on app rerun
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])
        if "sources" in message and message["sources"]:
            with st.expander("Show Sources"):
                for i, src in enumerate(message["sources"]):
                    st.json(src) # Display source dictionary as JSON for better readability

# Input box for new question
question = st.chat_input("Type your question here...")

# Handle new question submission
if question:
    st.session_state.messages.append({"role": "user", "content": question})
    with st.chat_message("user"):
        st.markdown(question)

    with st.chat_message("assistant"):
        with st.spinner("Searching and generating response..."):
            if vector_store is None or llm is None:
                st.error("RAG components failed to initialize. Please check the logs above.")
                response_content = "I'm sorry, the RAG system is not fully initialized. Please contact support."
                sources_to_display = []
            else:
                context_texts, source_documents = retrieve_chunks(question, vector_store)

                if not context_texts:
                    response_content = "I could not find relevant information in the knowledge base to answer your question."
                    sources_to_display = []
                else:
                    response_content = generate_response(llm, prompt_template, question, context_texts)
                    sources_to_display = source_documents

            st.markdown(response_content)
            if sources_to_display:
                with st.expander("Show Sources"):
                    for i, src in enumerate(sources_to_display):
                        st.json(src) # Display source dictionary as JSON for better readability

        st.session_state.messages.append({
            "role": "assistant",
            "content": response_content,
            "sources": sources_to_display
        })

# Clear Chat button
col1, col2 = st.columns([1, 5])
with col1:
    if st.button("Clear Chat"):
        st.session_state.messages = []
        st.experimental_rerun() # Rerun to clear the chat display

Writing app.py


In [ ]:
# Cell 4: Run Streamlit Application with ngrok
# Install ngrok
!pip install -q pyngrok

# --- ADD THIS LINE HERE ---
# Authenticate ngrok (replace YOUR_AUTHTOKEN_HERE_PASTE_YOUR_ACTUAL_TOKEN_HERE_FROM_NGROK_DASHBOARD with your actual ngrok authtoken)
# You can get a free authtoken from https://dashboard.ngrok.com/signup
!ngrok authtoken 2zfmBtFnyoMTY7WVqcWKRKafIww_6qNMxT2BpAWc3eJoXPaFA
# -------------------------

import subprocess
import time
from pyngrok import ngrok

# Start Streamlit in the background
print("Starting Streamlit app in the background...")
# Use nohup to run in background, redirect output to a file
# Adjust the command if your app.py is in a subdirectory (it shouldn't be with %%writefile)
streamlit_proc = subprocess.Popen(
    ["nohup", "streamlit", "run", "app.py", "&"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    preexec_fn=os.setsid
)
# Give Streamlit a moment to start
time.sleep(5)

print("Opening ngrok tunnel...")
# Ensure any previous ngrok tunnels are killed
ngrok.kill()
# Open a new tunnel to the Streamlit port (8501 is default for Streamlit)
public_url = ngrok.connect(8501)

print(f"Your Streamlit app is running at: {public_url}")
print("\nAccess the URL above in your browser.")
print("To stop the Streamlit app and ngrok tunnel, interrupt this cell (Ctrl+C).")

# Keep the cell running to maintain the tunnel
# You might want to periodically check if Streamlit process is still alive
try:
    while True:
        # Check Streamlit process status
        poll = streamlit_proc.poll()
        if poll is not None:
            print(f"Streamlit process exited with code {poll}.")
            stdout, stderr = streamlit_proc.communicate()
            print("Streamlit Stdout:\n", stdout.decode())
            print("Streamlit Stderr:\n", stderr.decode())
            break
        time.sleep(10) # Check every 10 seconds
except KeyboardInterrupt:
    print("\nInterrupting execution. Closing ngrok tunnel and Streamlit app...")
finally:
    ngrok.disconnect(public_url)
    ngrok.kill()
    if streamlit_proc.poll() is None: # If process is still running
        streamlit_proc.terminate()
        streamlit_proc.wait(5) # Give it 5 seconds to terminate
        if streamlit_proc.poll() is None: # If still alive, kill it
            streamlit_proc.kill()
    print("Cleanup complete.")

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Starting Streamlit app in the background...
Opening ngrok tunnel...
Your Streamlit app is running at: NgrokTunnel: "https://e13bbea62f77.ngrok-free.app" -> "http://localhost:8501"

Access the URL above in your browser.
To stop the Streamlit app and ngrok tunnel, interrupt this cell (Ctrl+C).
